Todo: Clean up and rename stuff

In [34]:
import scr_utils as scru
rate_sek=scru.get_sek()

In [35]:
import requests
from bs4 import BeautifulSoup

In [36]:
import pandas as pd
import json
import numpy as np

In [37]:
gameName='Hitman 3'
gameNameAlt='Hitman 3'

---- allkeyshop search----- 

In [38]:
url_endpoint = "https://www.allkeyshop.com/blog/wp-admin/admin-ajax.php"
reqParams = {
    'action': 'catalogue_query',
    'query[search]': gameName,
    'query[productsPerPage]': 16,
    'query[currency]': 'eur',
    'query[category]': 'pc-games-all'
}

resp = requests.get(url_endpoint, params=reqParams).json()['results']

In [39]:
soup = BeautifulSoup(resp, 'html.parser')
productId = soup.find(class_="metacritic-button")['data-product-id']
productId

'38888'

In [40]:
reqParams = {
    'action': 'get_offers',
    'product': productId,
    'currency': 'eur',
    'use_beta_offers_display': 1,
    'region':2
}

payload = {}

response = requests.request(
    "GET", url_endpoint, params=reqParams, data=payload)
#response.json()

In [41]:
file=response.json()

In [42]:
file.keys()

dict_keys(['success', 'offers', 'merchants', 'editions', 'regions'])

In [43]:
file['offers'][0]['price']['eur'].keys()

dict_keys(['bestCoupon', 'currency', 'price', 'priceWithoutCoupon'])

In [44]:
file['offers'][0]['price']['eur']['priceWithoutCoupon']

19.98

In [45]:
data=pd.json_normalize(file,record_path='offers')
data

,id,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,price.eur.currency,price.eur.price,price.eur.priceWithoutCoupon,price.eur.bestCoupon
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,18.38,19.98,NaN
1,133071217,https://www.gamivo.com/product/hitman-3?glv=ki...,True,218,1,2,InStock,steam,8%: AKSpooktober,8.00,%,False,eur,18.40,20.00,NaN
2,134533054,https://www.cjs-cdkeys.com/products/HITMAN-3-S...,True,67,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,21.09,21.09,NaN
3,134652039,https://www.instant-gaming.com/en/12619-/?igr=...,True,13,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,23.77,23.77,NaN
4,134524098,https://www.kinguin.net/category/115956/hitman...,True,47,1,2,InStock,steam,12%: AKS12,12.00,%,False,eur,24.04,27.32,NaN
5,134569366,https://www.cdkeys.com/hitman-3-pc-steam?mw_ar...,True,9,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,25.79,25.79,NaN
6,134524473,https://www.hrkgame.com/en/games/product/hitma...,True,94,1,2,InStock,steam,-10%: ALLKEYSHOP10,10.00,%,False,eur,26.16,29.07,NaN
7,134597584,https://k4g.com/product/hitman-3-steam-global-...,True,324,1,2,InStock,steam,14%: AKS14PUMPKIN,14.00,%,False,eur,26.88,31.26,NaN
8,134451227,https://royalcdkeys.com/products/hitman-3-stea...,True,321,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,26.94,26.94,NaN
9,134518514,https://www.g2a.com/hitman-3-pc-steam-key-glob...,True,61616,1,2,InStock,steam,1.99€ Monthly Sub,0.00,%,False,eur,28.20,28.20,NaN


mapp=data.drop(['price.eur.bestCoupon.discountValue','price.eur.bestCoupon.discountStrategy','price.eur.bestCoupon.isCashback','price.eur.bestCoupon.code', 'price.eur.bestCoupon'], axis=1,inplace=True)

In [46]:
data.rename(columns = {'price.eur.currency':'currency', 'price.eur.price':'price','price.eur.priceWithoutCoupon':'priceWithoutCoupon'}, inplace = True)

In [47]:
data[data.priceWithoutCoupon==data.priceWithoutCoupon.min()]

,id,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,priceWithoutCoupon,price.eur.bestCoupon
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,18.38,19.98,NaN


---------- dataframe for merchants ------------

In [48]:
merchants_list=pd.DataFrame(data=file['merchants']).T
merchants_list

,id,name,aggregateRating,types,paymentMethods,logoSlug,reviewUrl
272,272,Eneba,"{'value': 3.62, 'count': 314}",CD Key Store,"{'paymentMethode': ['card', 'paypal']}",eneba,https://www.allkeyshop.com/blog/review/eneba/
218,218,GAMIVO,"{'value': 4.18, 'count': 1250}",CD Key Store,"{'paymentMethode': ['card', 'paypal', 'crypto']}",gamivo,https://www.allkeyshop.com/blog/review/gamivo/
67,67,CJS-CDKeys,"{'value': 3.81, 'count': 819}",CD Key Store,"{'paymentMethode': ['card', 'paypal']}",cjs-cdkeys,https://www.allkeyshop.com/blog/review/cjs-cdk...
13,13,Instant Gaming,"{'value': 4.25, 'count': 307}",CD Key Store,"{'paymentMethode': ['paypal', 'card']}",instant-gaming,https://www.allkeyshop.com/blog/review/instant...
47,47,Kinguin,"{'value': 4.17, 'count': 588}",CD Key Store,"{'paymentMethode': ['card', 'paypal']}",kinguin,https://www.allkeyshop.com/blog/review/kinguin/
9,9,CDKeys.com,"{'value': 4.17, 'count': 724}",CD Key Store,"{'paymentMethode': ['card', 'paypal']}",cdkeys-com,https://www.allkeyshop.com/blog/review/cdkeys_...
94,94,HRK,"{'value': 2.71, 'count': 321}",CD Key Store,"{'paymentMethode': ['card', 'paypal', 'crypto']}",hrk,https://www.allkeyshop.com/blog/review/hrkgame/
324,324,K4G,"{'value': 4.84, 'count': 675}",CD Key Store,"{'paymentMethode': ['card', 'paypal']}",k4g,https://www.allkeyshop.com/blog/review/k4g/
321,321,Royal CD Keys,"{'value': 4.4, 'count': 290}",CD Key Store,{'paymentMethode': ['card']},royal-cd-keys,https://www.allkeyshop.com/blog/review/royalcd...
61616,61616,G2A Plus,"{'value': 4.29, 'count': 292}",CD Key Store,"{'paymentMethode': ['card', 'paypal']}",g2a,https://www.allkeyshop.com/blog/review/g2a/


---- merge merchant names with games hits to make it a bit clearer -----

In [49]:
merged_list = pd.merge(data,merchants_list[['id','name']],left_on='merchant',right_on='id',how='left')
merged_list.drop(['id_y','price'], axis=1,inplace=True)

merged_list.rename(columns = {'priceWithoutCoupon':'price'}, inplace = True)
#merged_list.set_index('id_x',inplace=True)

In [50]:
merged_list.head(-12)

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,price.eur.bestCoupon,name
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,19.98,NaN,Eneba
1,133071217,https://www.gamivo.com/product/hitman-3?glv=ki...,True,218,1,2,InStock,steam,8%: AKSpooktober,8.00,%,False,eur,20.00,NaN,GAMIVO
2,134533054,https://www.cjs-cdkeys.com/products/HITMAN-3-S...,True,67,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,21.09,NaN,CJS-CDKeys
3,134652039,https://www.instant-gaming.com/en/12619-/?igr=...,True,13,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,23.77,NaN,Instant Gaming
4,134524098,https://www.kinguin.net/category/115956/hitman...,True,47,1,2,InStock,steam,12%: AKS12,12.00,%,False,eur,27.32,NaN,Kinguin
5,134569366,https://www.cdkeys.com/hitman-3-pc-steam?mw_ar...,True,9,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,25.79,NaN,CDKeys.com
6,134524473,https://www.hrkgame.com/en/games/product/hitma...,True,94,1,2,InStock,steam,-10%: ALLKEYSHOP10,10.00,%,False,eur,29.07,NaN,HRK
7,134597584,https://k4g.com/product/hitman-3-steam-global-...,True,324,1,2,InStock,steam,14%: AKS14PUMPKIN,14.00,%,False,eur,31.26,NaN,K4G
8,134451227,https://royalcdkeys.com/products/hitman-3-stea...,True,321,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,26.94,NaN,Royal CD Keys
9,134518514,https://www.g2a.com/hitman-3-pc-steam-key-glob...,True,61616,1,2,InStock,steam,1.99€ Monthly Sub,0.00,%,False,eur,28.20,NaN,G2A Plus


In [51]:
instock_list=merged_list.loc[(merged_list['platform'] == 'steam') & (merged_list['stock'] == 'InStock')]

In [52]:
instock_list['price'].describe()

count    23.000000
mean     38.636087
std      16.687949
min      19.980000
25%      27.130000
50%      32.920000
75%      48.790000
max      89.990000
Name: price, dtype: float64

In [53]:
table=instock_list[["name", "price"]].groupby("name").min()
table.sort_values('price', ascending=True,inplace=True)

In [54]:
table.head(5)

,price
name,
Eneba,19.98
GAMIVO,20.00
CJS-CDKeys,21.09
Instant Gaming,23.77
CDKeys.com,25.79


In [55]:
instock_list['price'].min()

19.98

In [56]:
instock_list[instock_list.price==instock_list.price.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,price.eur.bestCoupon,name
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,19.98,NaN,Eneba


In [57]:
instock_list[instock_list.price==instock_list.price.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,price.eur.bestCoupon,name
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,19.98,NaN,Eneba


In [58]:
allkeysSE = instock_list.assign(priceSEK=instock_list["price"]*rate_sek)
#df2.round({'priceSEK': 1})
allkeysSE.priceSEK=allkeysSE.priceSEK.round(2)
allkeysSE.sort_values(by='priceSEK')

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,price.eur.bestCoupon,name,priceSEK
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,19.98,NaN,Eneba,218.68
1,133071217,https://www.gamivo.com/product/hitman-3?glv=ki...,True,218,1,2,InStock,steam,8%: AKSpooktober,8.00,%,False,eur,20.00,NaN,GAMIVO,218.90
2,134533054,https://www.cjs-cdkeys.com/products/HITMAN-3-S...,True,67,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,21.09,NaN,CJS-CDKeys,230.83
3,134652039,https://www.instant-gaming.com/en/12619-/?igr=...,True,13,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,23.77,NaN,Instant Gaming,260.16
5,134569366,https://www.cdkeys.com/hitman-3-pc-steam?mw_ar...,True,9,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,25.79,NaN,CDKeys.com,282.27
8,134451227,https://royalcdkeys.com/products/hitman-3-stea...,True,321,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,26.94,NaN,Royal CD Keys,294.85
4,134524098,https://www.kinguin.net/category/115956/hitman...,True,47,1,2,InStock,steam,12%: AKS12,12.00,%,False,eur,27.32,NaN,Kinguin,299.01
9,134518514,https://www.g2a.com/hitman-3-pc-steam-key-glob...,True,61616,1,2,InStock,steam,1.99€ Monthly Sub,0.00,%,False,eur,28.20,NaN,G2A Plus,308.64
6,134524473,https://www.hrkgame.com/en/games/product/hitma...,True,94,1,2,InStock,steam,-10%: ALLKEYSHOP10,10.00,%,False,eur,29.07,NaN,HRK,318.17
11,134518513,https://www.g2a.com/hitman-3-pc-steam-key-glob...,True,61,1,2,InStock,steam,NaN,NaN,NaN,NaN,eur,30.65,NaN,G2A,335.46


In [59]:
allkeysSE.loc[allkeysSE['merchant']=='13',['priceSEK','name']].sort_values(by='priceSEK').head(3)

,priceSEK,name
3,260.16,Instant Gaming


In [60]:
allkeysSE[allkeysSE.priceSEK==allkeysSE.priceSEK.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,price.eur.bestCoupon,name,priceSEK
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,19.98,NaN,Eneba,218.68


In [61]:
allkeysSE.loc[allkeysSE['priceSEK'].idxmin()]

id_x                                                                             134513555
affiliateUrl                             https://www.eneba.com/steam-hitman-3-pc-steam-...
isActive                                                                              True
merchant                                                                               272
edition                                                                                  1
region                                                                                   2
stock                                                                              InStock
platform                                                                             steam
price.eur.bestCoupon.code                                                         8%: AKS8
price.eur.bestCoupon.discountValue                                                    8.00
price.eur.bestCoupon.discountStrategy                                                    %

In [62]:
allkeysSE.nsmallest(3,'priceSEK').filter(['name','priceSEK'],axis=1)

,name,priceSEK
0,Eneba,218.68
1,GAMIVO,218.90
2,CJS-CDKeys,230.83


In [63]:
allkeysSE.loc[allkeysSE['platform']=='steam',['priceSEK','name']].sort_values(by='priceSEK').head(3)

,priceSEK,name
0,218.68,Eneba
1,218.90,GAMIVO
2,230.83,CJS-CDKeys


In [64]:
allkeysSE[allkeysSE.price==allkeysSE.price.min()]

,id_x,affiliateUrl,isActive,merchant,edition,region,stock,platform,price.eur.bestCoupon.code,price.eur.bestCoupon.discountValue,price.eur.bestCoupon.discountStrategy,price.eur.bestCoupon.isCashback,currency,price,price.eur.bestCoupon,name,priceSEK
0,134513555,https://www.eneba.com/steam-hitman-3-pc-steam-...,True,272,1,2,InStock,steam,8%: AKS8,8.00,%,False,eur,19.98,NaN,Eneba,218.68


In [65]:
scru.to_paraquet('allkey_prices',allkeysSE)

pyarrow.Table
id_x: int64
affiliateUrl: string
isActive: bool
merchant: string
edition: string
region: string
stock: string
platform: string
price.eur.bestCoupon.code: string
price.eur.bestCoupon.discountValue: string
price.eur.bestCoupon.discountStrategy: string
price.eur.bestCoupon.isCashback: bool
currency: string
price: double
price.eur.bestCoupon: double
name: string
priceSEK: double
__index_level_0__: int64
----
id_x: [[134513555,133071217,134533054,134652039,134524098,...,134514503,134523817,134523816,134658897,134658895]]
affiliateUrl: [["https://www.eneba.com/steam-hitman-3-pc-steam-key-global?af_id=allkeyshop","https://www.gamivo.com/product/hitman-3?glv=kiwhuamu","https://www.cjs-cdkeys.com/products/HITMAN-3-Steam-Key.html","https://www.instant-gaming.com/en/12619-/?igr=289098","https://www.kinguin.net/category/115956/hitman-3-steam-cd-key?r=3445&nosalesbooster=1&currency=EUR&discount=AKS12",...,"https://www.eneba.com/steam-hitman-3-deluxe-edition-pc-steam-key-global?af_id=a